In [1]:
! pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 21.4 MB/s eta 0:00:00


In [2]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
! pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.6 MB/s eta 0:00:00


In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def load_and_embedd_dataset(
        dataset_name: str = 'ErikCikalleshi/new_york_times_news_2000_2007',
        split: str = 'train',
        model: SentenceTransformer = SentenceTransformer('paraphrase-MiniLM-L6-v2'),
        text_field: str = 'content',
        rec_num: int = 400
) -> tuple:
    """
    Load a dataset and embedd the text field using a sentence-transformer model

    """
    from datasets import load_dataset

    print("Loading and embedding the dataset")

    dataset = load_dataset(dataset_name, split=split)
    embeddings = model.encode(dataset[text_field][:rec_num])

    print("Done!")
    return dataset, embeddings

In [6]:
DATASET_NAME = 'ErikCikalleshi/new_york_times_news_2000_2007'

dataset, embeddings = load_and_embedd_dataset(
    dataset_name=DATASET_NAME,
    rec_num=400,
    model=model,
)
shape = embeddings.shape

Loading and embedding the dataset


README.md:   0%|          | 0.00/473 [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/239M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/238M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

...

train-00005-of-00006.parquet:   0%|          | 0.00/238M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/160M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/497249 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/55250 [00:00<?, ? examples/s]

Done!


In [7]:
def create_faiss_index(embeddings: np.ndarray, nlist: int = 100):
    """
    Create and train a FAISS index from embeddings using Approximate Nearest Neighbors (ANN), where nlist is
    the number of clusters to create for the ANN search.

    """
    dimension = embeddings.shape[1]
    quantizer = faiss.IndexFlatL2(dimension)
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

    index.train(embeddings.astype(np.float32))
    index.add(embeddings.astype(np.float32))
    return index

In [8]:
faiss_index = create_faiss_index(embeddings)

In [9]:
def augment_prompt(
        query: str,
        model: SentenceTransformer = SentenceTransformer('paraphrase-MiniLM-L6-v2'),
        index=None,
        dataset: dict = None,
        text_field: str = 'content',
) -> str:
    """
    Augment the prompt with the top 3 results from the knowledge base

    """
    query_embedding = model.encode(query).astype('float32').reshape(1, -1)
    distances, indices = index.search(query_embedding, k=3)
    text_matches = [dataset[text_field][i] for i in indices[0]]

    source_knowledge = "\n\n".join(text_matches)

    augmented_prompt = f"""Using the contexts below, answer the query.
    Contexts:
    {source_knowledge}
    If the answer is not included in the source knowledge - say that you don't know.
    Query: {query}"""

    return augmented_prompt, source_knowledge


In [10]:
with open("chohere_api_keys.txt") as f:
    COHERE_API_KEY = f.read().strip()

**Experiments and Results:**

Query1: How many jobs were added on Long Island in 2001?

*LLM's answer:*

In [11]:
import cohere

query = "How many jobs were added on Long Island in 2001?"

co = cohere.Client(api_key=COHERE_API_KEY)
response = co.chat(
        model='command-r-plus',
        message=query,
    )
response.text

'According to data from the New York State Department of Labor, Long Island added 11,500 jobs in 2001. This represented a growth rate of 1.1%, which was slightly lower than the statewide growth rate of 1.2% for the same year. The job growth on Long Island was driven by gains in the education, health, and social services sectors, which added a total of 7,700 jobs. Other sectors that contributed to the job growth include professional and business services, leisure and hospitality, and government.'

*Our RAG-ANN answer:*

In [12]:
query = "How many jobs were added on Long Island in 2001?"
augmented_prompt, source_knowledge = augment_prompt(query, model=model, index=faiss_index, dataset=dataset)

response = co.chat(
        model='command-r-plus',
        message=augmented_prompt,
    )
print(response.text)


6,700 jobs were added on Long Island in 2001.


**Evaluation using reformulation of the query**

In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

def generate_reformulations(query, cohere_client, num_reformulations=5):
    """
    Generate reformulations of the query using an LLM (in our case, Cohere).
    """
    reformulations = []
    for _ in range(num_reformulations):
        response = cohere_client.chat(model='command-r-plus', message=f"Please reformulate: {query}")
        reformulations.append(response.text)
    return reformulations

def evaluate_rag_with_reformulations(query, cohere_client, rag_model, dataset, faiss_index, num_reformulations=5):
    """
    Evaluate the RAG model by generating reformulations of the query, retrieving responses,
    and calculating similarity between responses.
    """
    original_augmented_prompt, original_source_knowledge = augment_prompt(query, model=rag_model, index=faiss_index, dataset=dataset)
    original_response = cohere_client.chat(model='command-r-plus', message=original_augmented_prompt).text

    reformulated_queries = generate_reformulations(query, cohere_client, num_reformulations)

    responses = [original_response]
    for reformulated_query in reformulated_queries:
        reformulated_augmented_prompt, reformulated_source_knowledge = augment_prompt(
            reformulated_query, model=rag_model, index=faiss_index, dataset=dataset
        )
        reformulated_response = cohere_client.chat(model='command-r-plus', message=reformulated_augmented_prompt).text
        responses.append(reformulated_response)

    response_embeddings = model.encode(responses)
    similarity_matrix = cosine_similarity(response_embeddings)

    similarities_to_original = similarity_matrix[0, 1:]
    average_similarity = np.mean(similarities_to_original)

    print("Original Response:", original_response)
    print("\nReformulated Responses and Similarities:")
    for i, (reform_query, reform_response, sim) in enumerate(zip(reformulated_queries, responses[1:], similarities_to_original)):
        print(f"\nReformulation {i+1}: {reform_query}")
        print(f"Response: {reform_response}")
        print(f"Similarity to Original: {sim:.4f}")

    print(f"\nAverage Similarity to Original Response: {average_similarity:.4f}")
    return average_similarity



In [14]:
query = "How many jobs were added on Long Island in 2001?"

average_similarity = evaluate_rag_with_reformulations(query, cohere_client=co, rag_model=model, dataset=dataset, faiss_index=faiss_index, num_reformulations=5)
print("Evaluation completed. Average similarity score:", average_similarity)

Original Response: 6,700 jobs were added on Long Island in 2001.

Reformulated Responses and Similarities:

Reformulation 1: What was the net change in employment on Long Island in the year 2001?
Response: I don't know. The text provided mentions that Long Island added about 6,700 jobs last year and that unemployment rose to 4.4 percent in January, up from 3.1 percent in January 2001. However, it does not explicitly state the net change in employment for the entire year of 2001.
Similarity to Original: 0.6632

Reformulation 2: What was the net change in jobs on Long Island in the year 2001?
Response: The net change in jobs on Long Island in the year 2001 was an addition of 6,700 jobs.
Similarity to Original: 0.8898

Reformulation 3: What was the net change in jobs on Long Island in the year 2001?
Response: The net change in jobs on Long Island in 2001 was an increase of 6,700 jobs.
Similarity to Original: 0.8707

Reformulation 4: What was the net increase in jobs on Long Island in 2001

**Evaluation using semantic coherence**

In [15]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def compute_coherence(query, response, model):
    """
    Compute the semantic coherence between the query and the response
    using cosine similarity.

    """
    query_embedding = model.encode(query).reshape(1, -1)
    response_embedding = model.encode(response).reshape(1, -1)

    coherence_score = cosine_similarity(query_embedding, response_embedding)[0][0]

    return coherence_score


In [16]:
query = "How many jobs were added on Long Island in 2001?"

augmented_prompt, source_knowledge = augment_prompt(query, model=model, index=faiss_index, dataset=dataset)
response = co.chat(model='command-r-plus', message=augmented_prompt).text

coherence_score = compute_coherence(query, response, model)
print("Semantic Coherence Score:", coherence_score)

Semantic Coherence Score: 0.79949594


Evaluating our Vanilla RAG:

In [26]:
def compute_coherence_original(query, response, model):
    """
    Compute the semantic coherence between the query and the response of a classic RAG
    using cosine similarity.

    """

    co = cohere.Client(api_key=COHERE_API_KEY)
    response = co.chat(
            model='command-r-plus',
            message=query,
        )

    query_embedding = model.encode(query).reshape(1, -1)
    response_embedding = model.encode(response.text).reshape(1, -1)

    coherence_score = cosine_similarity(query_embedding, response_embedding)[0][0]

    return coherence_score


In [27]:
query = "How many jobs were added on Long Island in 2001?"
coherence_score = compute_coherence_original(query, response, model)
print("Semantic Coherence Score:", coherence_score)

Semantic Coherence Score: 0.7572492


Query 2: In 2003 Turkey were waiting for the Bush administration to answer their demand for an economic aid package to ensure their participation in a war with Iraq. What was the amount of the economic aid package?



In [28]:
query = "In 2003 Turkey were waiting for the Bush administration to answer their demand for an economic aid package to ensure their participation in a war with Iraq. What was the amount of the economic aid package?"

average_similarity = evaluate_rag_with_reformulations(query, cohere_client=co, rag_model=model, dataset=dataset, faiss_index=faiss_index, num_reformulations=5)
print("Evaluation completed. Average similarity score:", average_similarity)

Original Response: The amount of the economic aid package that Turkey was waiting for the Bush administration to answer their demand for in 2003 was $32 billion.

Reformulated Responses and Similarities:

Reformulation 1: In 2003, Turkey awaited a response from the Bush administration regarding their request for economic assistance. The country had asked for an economic aid package as a condition for their involvement in the impending Iraq War. At the time, the specifics of the requested package, including the total amount of financial assistance sought by Turkey, were not publicly disclosed and were subject to negotiation between the two countries. 

While the exact amount of the economic aid package is not mentioned in your original statement, it is clear that Turkey considered this assistance vital to their decision to participate in the war. The negotiation process would likely have involved discussions on the scope and nature of the aid, with the final amount being determined base

In [29]:
augmented_prompt, source_knowledge = augment_prompt(query, model=model, index=faiss_index, dataset=dataset)
response = co.chat(model='command-r-plus', message=augmented_prompt).text

coherence_score = compute_coherence(query, response, model)
print("Semantic Coherence Score:", coherence_score)

Semantic Coherence Score: 0.8546722


Evaluation of our Vanilla RAG:

In [30]:
coherence_score = compute_coherence_original(query, response, model)
print("Semantic Coherence Score:", coherence_score)

Semantic Coherence Score: 0.79856193


Query 3: How much was the prize for the winners of the first Westchester Prize for New Works? Among how much Westchester arts organizations was it shared equally?

In [31]:
query = "How much was the prize for the winners of the first Westchester Prize for New Works? Among how much Westchester arts organizations was it shared equally?"

average_similarity = evaluate_rag_with_reformulations(query, cohere_client=co, rag_model=model, dataset=dataset, faiss_index=faiss_index, num_reformulations=5)
print("Evaluation completed. Average similarity score:", average_similarity)

Original Response: The prize for the winners of the first Westchester Prize for New Works was $50,000, and it was shared equally among three Westchester arts organizations.

Reformulated Responses and Similarities:

Reformulation 1: How much was the prize for the winner of the first Westchester Prize for New Works, and among how many Westchester arts organizations was it divided equally?
Response: The prize for the winner of the first Westchester Prize for New Works was $50,000, and it was divided equally among three Westchester arts organizations.
Similarity to Original: 0.9853

Reformulation 2: How much was the prize for the winners of the inaugural Westchester Prize for New Works, and between how many Westchester arts organizations was the prize money shared equally?
Response: The prize money for the winners of the inaugural Westchester Prize for New Works was $50,000, and it was shared equally among three Westchester arts organizations and their artistic partners.
Similarity to Ori

In [32]:
augmented_prompt, source_knowledge = augment_prompt(query, model=model, index=faiss_index, dataset=dataset)
response = co.chat(model='command-r-plus', message=augmented_prompt).text

coherence_score = compute_coherence(query, response, model)
print("Semantic Coherence Score:", coherence_score)

Semantic Coherence Score: 0.818069


Evaluation of our Vanilla RAG:

In [33]:
coherence_score = compute_coherence_original(query, response, model)
print("Semantic Coherence Score:", coherence_score)

Semantic Coherence Score: 0.8424914


Query 4:Who is Edith Friedman ?

In [ ]:
query = "Who is Edith Friedman ?"

In [35]:
augmented_prompt, source_knowledge = augment_prompt(query, model=model, index=faiss_index, dataset=dataset)
response = co.chat(model='command-r-plus', message=augmented_prompt).text

coherence_score = compute_coherence(query, response, model)
print("Semantic Coherence Score:", coherence_score)

Semantic Coherence Score: 0.08450343
